# 09 - Large Models on Kaggle (llamatelemetry) - e3

Objective: Download three GGUF models from Hugging Face and run them one-by-one on Kaggle T4 GPU 0, executing all steps after Step 4 for each model.


## Step 0.5: Verify Dual GPU Environment


In [1]:
import subprocess

print('='*70)
print('?? GPU ENVIRONMENT CHECK')
print('='*70)

result = subprocess.run(
    ['nvidia-smi', '--query-gpu=index,name,memory.total,memory.free,compute_cap', '--format=csv,noheader'],
    capture_output=True, text=True
)
print(result.stdout)


?? GPU ENVIRONMENT CHECK
0, Tesla T4, 15360 MiB, 14913 MiB, 7.5
1, Tesla T4, 15360 MiB, 14913 MiB, 7.5



## Step 1: Install llamatelemetry and Dependencies


In [2]:
%%time
!pip install -q --no-cache-dir --force-reinstall git+https://github.com/llamatelemetry/llamatelemetry.git@v0.1.0
!pip install -q huggingface_hub sseclient-py

import llamatelemetry
print('? llamatelemetry {} installed'.format(llamatelemetry.__version__))


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 553.2/553.2 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 193.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 307.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 298.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.0/220.0 kB 365.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 331.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 332.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.9/152.9 kB 312.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.5/153.5 kB 342.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.8/20


🎯 llamatelemetry v0.1.0 First-Time Setup - Kaggle 2× T4 Multi-GPU

🎮 GPU Detected: Tesla T4 (Compute 7.5)
  ✅ Tesla T4 detected - Perfect for llamatelemetry v0.1.0!
🌐 Platform: Colab

📦 Downloading Kaggle 2× T4 binaries (~961 MB)...
    Features: FlashAttention + Tensor Cores + Multi-GPU tensor-split

➡️  Attempt 1: HuggingFace (llamatelemetry-v0.1.0-cuda12-kaggle-t4x2.tar.gz)
📥 Downloading v0.1.0 from HuggingFace Hub...
   Repo: waqasm86/llamatelemetry-binaries
   File: v0.1.0/llamatelemetry-v0.1.0-cuda12-kaggle-t4x2.tar.gz


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_validators.py:202: UserWarning: The `local_dir_use_symlinks` argument is deprecated and ignored in `hf_hub_download`. Downloading to a local directory does not use symlinks anymore.
  warnings.warn(


v0.1.0/llamatelemetry-v0.1.0-cuda12-kagg(…):   0%|          | 0.00/1.40G [00:00<?, ?B/s]

🔐 Verifying SHA256 checksum...
   ✅ Checksum verified
📦 Extracting llamatelemetry-v0.1.0-cuda12-kaggle-t4x2.tar.gz...
Found 21 files in archive
Extracted 21 files to /root/.cache/llamatelemetry/extract_0.1.0
✅ Extraction complete!
  Found bin/ and lib/ under /root/.cache/llamatelemetry/extract_0.1.0/llamatelemetry-v0.1.0-cuda12-kaggle-t4x2
  Copied 13 binaries to /usr/local/lib/python3.12/dist-packages/llamatelemetry/binaries/cuda12
  Copied 2 libraries to /usr/local/lib/python3.12/dist-packages/llamatelemetry/lib
✅ Binaries installed successfully!

? llamatelemetry 0.1.0 installed
CPU times: user 49 s, sys: 11.4 s, total: 1min
Wall time: 1min 57s


## Step 2: Setup Secrets (Hugging Face + Graphistry Optional)


In [4]:
from kaggle_secrets import UserSecretsClient
from huggingface_hub import login

user_secrets = UserSecretsClient()

hf_token = None
try:
    hf_token = user_secrets.get_secret('HF_TOKEN_2')
except Exception:
    pass

if hf_token:
    login(token=hf_token)
    print('? Hugging Face login OK')
else:
    print('?? HF_TOKEN not found. Public models will still download.')


? Hugging Face login OK


## Step 3: Download 3 GGUF Models (1?4 GB each)


In [5]:
from huggingface_hub import hf_hub_download
import os

print('='*70)
print('?? DOWNLOADING 3 GGUF MODELS')
print('='*70)

os.makedirs('/kaggle/working/models', exist_ok=True)

model_configs = [
    {
        'name': 'Llama-3.2-3B-Instruct',
        'repo': 'bartowski/Llama-3.2-3B-Instruct-GGUF',
        'file': 'Llama-3.2-3B-Instruct-Q4_K_M.gguf',
        'size_gb': 1.88
    },
    {
        'name': 'Mistral-7B-Instruct',
        'repo': 'TheBloke/Mistral-7B-Instruct-v0.2-GGUF',
        'file': 'mistral-7b-instruct-v0.2.Q4_K_M.gguf',
        'size_gb': 4.08
    },
    {
        'name': 'Phi-3-mini-4k',
        'repo': 'microsoft/Phi-3-mini-4k-instruct-gguf',
        'file': 'Phi-3-mini-4k-instruct-q4.gguf',
        'size_gb': 2.10
    },
]

model_paths = {}
for model in model_configs:
    print('\n?? Downloading {}...'.format(model['name']))
    print('   Estimated size: {:.2f} GB'.format(model['size_gb']))
    try:
        model_path = hf_hub_download(
            repo_id=model['repo'],
            filename=model['file'],
            local_dir='/kaggle/working/models',
            local_dir_use_symlinks=False
        )
        actual_size = os.path.getsize(model_path) / (1024**3)
        model_paths[model['name']] = model_path
        print('? Downloaded: {} ({:.2f} GB)'.format(os.path.basename(model_path), actual_size))
    except Exception as e:
        print('?? Failed to download {}: {}'.format(model['name'], e))

print('\n?? Summary: {} models downloaded'.format(len(model_paths)))
for name, path in model_paths.items():
    print('   ? {}: {}'.format(name, os.path.basename(path)))


?? DOWNLOADING 3 GGUF MODELS

?? Downloading Llama-3.2-3B-Instruct...
   Estimated size: 1.88 GB


Llama-3.2-3B-Instruct-Q4_K_M.gguf:   0%|          | 0.00/2.02G [00:00<?, ?B/s]

? Downloaded: Llama-3.2-3B-Instruct-Q4_K_M.gguf (1.88 GB)

?? Downloading Mistral-7B-Instruct...
   Estimated size: 4.08 GB


mistral-7b-instruct-v0.2.Q4_K_M.gguf:   0%|          | 0.00/4.37G [00:00<?, ?B/s]

? Downloaded: mistral-7b-instruct-v0.2.Q4_K_M.gguf (4.07 GB)

?? Downloading Phi-3-mini-4k...
   Estimated size: 2.10 GB


Phi-3-mini-4k-instruct-q4.gguf:   0%|          | 0.00/2.39G [00:00<?, ?B/s]

? Downloaded: Phi-3-mini-4k-instruct-q4.gguf (2.23 GB)

?? Summary: 3 models downloaded
   ? Llama-3.2-3B-Instruct: Llama-3.2-3B-Instruct-Q4_K_M.gguf
   ? Mistral-7B-Instruct: mistral-7b-instruct-v0.2.Q4_K_M.gguf
   ? Phi-3-mini-4k: Phi-3-mini-4k-instruct-q4.gguf


## Step 4: Helper Functions


In [6]:
import time
import requests

def wait_until_ready(server_url, timeout=90):
    start = time.time()
    while time.time() - start < timeout:
        try:
            r = requests.get(server_url + '/health', timeout=2)
            if r.status_code == 200:
                return True
        except Exception:
            pass
        time.sleep(1)
    return False


## Steps 5?13: Run Each Model Sequentially (GPU 0)


In [7]:
from llamatelemetry.server import ServerManager
from llamatelemetry.api.client import LlamaCppClient
import subprocess
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '0'

results_summary = []

for model_name, model_path in model_paths.items():
    print('\n' + '='*80)
    print('?? RUNNING MODEL: {}'.format(model_name))
    print('='*80)

    print('\n?? Step 5: GPU Availability')
    result = subprocess.run(
        ['nvidia-smi', '--query-gpu=index,name,memory.total,memory.free', '--format=csv,noheader'],
        capture_output=True, text=True
    )
    print(result.stdout)

    print('?? Step 6: Start llama-server (GPU 0)')
    server = ServerManager(server_url='http://127.0.0.1:8090')
    try:
        server.start_server(
            model_path=model_path,
            host='127.0.0.1',
            port=8090,
            gpu_layers=99,
            tensor_split='1.0,0.0',
            ctx_size=2048,
            flash_attn='auto',
            timeout=90,
            verbose=True,
        )
        if not wait_until_ready('http://127.0.0.1:8090', timeout=90):
            print('? Server did not become ready')
            server.stop_server()
            continue
    except Exception as e:
        print('? Failed to start server: {}'.format(e))
        try:
            server.stop_server()
        except Exception:
            pass
        continue

    print('\n?? Step 7: GPU Memory Usage')
    subprocess.run(['nvidia-smi'])

    print('\n?? Step 8: Test Inference')
    client = LlamaCppClient(base_url='http://127.0.0.1:8090')
    prompt = 'Explain quantum computing in simple terms.'
    start = time.time()
    response = client.chat.create(
        messages=[{'role': 'user', 'content': prompt}],
        max_tokens=200,
        temperature=0.7,
    )
    elapsed = time.time() - start
    tokens = response.usage.completion_tokens if response.usage else 0
    print(response.choices[0].message.content)
    print('\n?? Inference: {:.2f}s | {} tokens | {:.1f} tok/s'.format(elapsed, tokens, (tokens/elapsed if elapsed>0 else 0)))

    print('\n?? Step 9: Performance Benchmarks')
    prompts = [
        'Hello',
        'What is the capital of France?',
        'Write a paragraph about machine learning.',
        'Explain the differences between Python and JavaScript with examples.'
    ]

    bench_times = []
    bench_speeds = []

    for p in prompts:
        t0 = time.time()
        r = client.chat.create(
            messages=[{'role': 'user', 'content': p}],
            max_tokens=80,
            temperature=0.7,
        )
        dt = time.time() - t0
        tk = r.usage.completion_tokens if r.usage else 0
        sp = tk/dt if dt>0 else 0
        bench_times.append(dt)
        bench_speeds.append(sp)
        print('   Prompt: {}... | {:.2f}s | {:.1f} tok/s'.format(p[:40], dt, sp))

    avg_time = sum(bench_times) / len(bench_times)
    avg_speed = sum(bench_speeds) / len(bench_speeds)

    print('\n?? Step 10: Streaming Test')
    stream_prompt = 'What are the benefits of streaming?'
    try:
        stream_resp = client.chat.create(
            messages=[{'role': 'user', 'content': stream_prompt}],
            max_tokens=80,
            temperature=0.7,
            stream=True
        )
        out = ''
        for chunk in stream_resp:
            if hasattr(chunk, 'choices') and chunk.choices:
                choice = chunk.choices[0]
                if hasattr(choice, 'delta') and choice.delta and getattr(choice.delta, 'content', None):
                    out += choice.delta.content
        print(out[:200])
    except Exception as e:
        print('?? Streaming error: {}'.format(e))

    print('\n?? Step 11: Stop server')
    server.stop_server()
    time.sleep(2)

    results_summary.append({
        'Model': model_name,
        'Avg Time (s)': '{:.2f}'.format(avg_time),
        'Avg Speed (tok/s)': '{:.1f}'.format(avg_speed),
    })

print('\n' + '='*80)
print('?? SUMMARY')
print('='*80)
if results_summary:
    import pandas as pd
    df = pd.DataFrame(results_summary)
    print(df.to_string(index=False))
else:
    print('No benchmark results.')



?? RUNNING MODEL: Llama-3.2-3B-Instruct

?? Step 5: GPU Availability
0, Tesla T4, 15360 MiB, 14913 MiB
1, Tesla T4, 15360 MiB, 14913 MiB

?? Step 6: Start llama-server (GPU 0)
GPU Check:
  Platform: kaggle
  GPU: Tesla T4
  Compute Capability: 7.5
  Status: ✓ Compatible
Starting llama-server...
  Executable: /usr/local/lib/python3.12/dist-packages/llamatelemetry/binaries/cuda12/llama-server
  Model: Llama-3.2-3B-Instruct-Q4_K_M.gguf
  GPU Layers: 99
  Context Size: 2048
  Server URL: http://127.0.0.1:8090
Waiting for server to be ready....... ✓ Ready in 4.1s

?? Step 7: GPU Memory Usage
Thu Feb  5 07:58:39 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 580.105.08             Driver Version: 580.105.08     CUDA Version: 13.0     |
+-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan

In [8]:
# Step 5: GPU Availability Check (run once before all models)
from llamatelemetry.server import ServerManager
from llamatelemetry.api.client import LlamaCppClient
import subprocess
import os
import time
import requests

# Set GPU 0
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

print('='*70)
print('?? STEP 5: GPU AVAILABILITY CHECK')
print('='*70)

result = subprocess.run(
    ['nvidia-smi', '--query-gpu=index,name,memory.total,memory.free', '--format=csv,noheader'],
    capture_output=True, text=True
)
print(result.stdout)

?? STEP 5: GPU AVAILABILITY CHECK
0, Tesla T4, 15360 MiB, 14913 MiB
1, Tesla T4, 15360 MiB, 14913 MiB



In [9]:
# Step 6: Start llama-server for each model (will run sequentially)
print('='*70)
print('?? STEP 6: START LLAMA-SERVER FOR EACH MODEL')
print('='*70)

# Helper function for waiting
def wait_until_ready(server_url, timeout=90):
    start = time.time()
    while time.time() - start < timeout:
        try:
            r = requests.get(server_url + '/health', timeout=2)
            if r.status_code == 200:
                return True
        except Exception:
            pass
        time.sleep(1)
    return False

# Dictionary to store running servers
servers = {}

for model_name, model_path in model_paths.items():
    print(f'\n?? Starting server for: {model_name}')
    print(f'   Model path: {os.path.basename(model_path)}')
    
    server = ServerManager(server_url='http://127.0.0.1:8090')
    try:
        server.start_server(
            model_path=model_path,
            host='127.0.0.1',
            port=8090,
            gpu_layers=99,
            tensor_split='1.0,0.0',
            ctx_size=2048,
            flash_attn='auto',
            timeout=90,
            verbose=True,
        )
        
        if wait_until_ready('http://127.0.0.1:8090', timeout=90):
            servers[model_name] = server
            print(f'? Server for {model_name} is ready')
        else:
            print(f'? Server for {model_name} did not become ready')
            server.stop_server()
    except Exception as e:
        print(f'? Failed to start server for {model_name}: {e}')

?? STEP 6: START LLAMA-SERVER FOR EACH MODEL

?? Starting server for: Llama-3.2-3B-Instruct
   Model path: Llama-3.2-3B-Instruct-Q4_K_M.gguf
GPU Check:
  Platform: kaggle
  GPU: Tesla T4
  Compute Capability: 7.5
  Status: ✓ Compatible
Starting llama-server...
  Executable: /usr/local/lib/python3.12/dist-packages/llamatelemetry/binaries/cuda12/llama-server
  Model: Llama-3.2-3B-Instruct-Q4_K_M.gguf
  GPU Layers: 99
  Context Size: 2048
  Server URL: http://127.0.0.1:8090
Waiting for server to be ready..... ✓ Ready in 2.0s
? Server for Llama-3.2-3B-Instruct is ready

?? Starting server for: Mistral-7B-Instruct
   Model path: mistral-7b-instruct-v0.2.Q4_K_M.gguf
GPU Check:
  Platform: kaggle
  GPU: Tesla T4
  Compute Capability: 7.5
  Status: ✓ Compatible
✓ llama-server already running at http://127.0.0.1:8090
? Server for Mistral-7B-Instruct is ready

?? Starting server for: Phi-3-mini-4k
   Model path: Phi-3-mini-4k-instruct-q4.gguf
GPU Check:
  Platform: kaggle
  GPU: Tesla T4
  Compu

In [10]:
# Step 7: GPU Memory Usage after loading each model
print('='*70)
print('?? STEP 7: GPU MEMORY USAGE AFTER LOADING MODELS')
print('='*70)

for model_name in model_paths.keys():
    if model_name in servers:
        print(f'\n?? GPU Memory Usage for {model_name}:')
        subprocess.run(['nvidia-smi'])

?? STEP 7: GPU MEMORY USAGE AFTER LOADING MODELS

?? GPU Memory Usage for Llama-3.2-3B-Instruct:
Thu Feb  5 08:09:36 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 580.105.08             Driver Version: 580.105.08     CUDA Version: 13.0     |
+-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   44C    P0             25W /   70W |    2333MiB /  15360MiB |      0%      Default |
|                                         |

In [11]:
# Step 8: Test Inference for each model
print('='*70)
print('?? STEP 8: TEST INFERENCE FOR EACH MODEL')
print('='*70)

inference_results = {}

for model_name, model_path in model_paths.items():
    if model_name not in servers:
        print(f'\n?? Skipping {model_name} - server not running')
        continue
    
    print(f'\n?? Testing inference for: {model_name}')
    
    client = LlamaCppClient(base_url='http://127.0.0.1:8090')
    prompt = 'Explain quantum computing in simple terms.'
    
    start = time.time()
    response = client.chat.create(
        messages=[{'role': 'user', 'content': prompt}],
        max_tokens=200,
        temperature=0.7,
    )
    elapsed = time.time() - start
    tokens = response.usage.completion_tokens if response.usage else 0
    
    print('Response preview:')
    print(response.choices[0].message.content[:200] + '...')
    print(f'\n?? Inference: {elapsed:.2f}s | {tokens} tokens | {tokens/elapsed if elapsed>0 else 0:.1f} tok/s')
    
    inference_results[model_name] = {
        'time': elapsed,
        'tokens': tokens,
        'speed': tokens/elapsed if elapsed>0 else 0
    }

?? STEP 8: TEST INFERENCE FOR EACH MODEL

?? Testing inference for: Llama-3.2-3B-Instruct
Response preview:
Quantum computing is a new way of processing information that's different from traditional computers.

**Traditional Computers**

Traditional computers use "bits" to store and process information. Bit...

?? Inference: 2.78s | 200 tokens | 71.9 tok/s

?? Testing inference for: Mistral-7B-Instruct
Response preview:
Quantum computing is a new way of processing information that's different from the computers we use today. Here are some simple explanations:

**Classical Computing vs Quantum Computing**

Imagine you...

?? Inference: 2.72s | 200 tokens | 73.5 tok/s

?? Testing inference for: Phi-3-mini-4k
Response preview:
Quantum computing is a way of processing information that's different from how regular computers work.

**Classical Computers**

Imagine you have a giant library with millions of books on shelves. Eac...

?? Inference: 2.72s | 200 tokens | 73.5 tok/s


In [12]:
# Step 9: Performance Benchmarks for each model
print('='*70)
print('?? STEP 9: PERFORMANCE BENCHMARKS')
print('='*70)

benchmark_results = {}

prompts = [
    'Hello',
    'What is the capital of France?',
    'Write a paragraph about machine learning.',
    'Explain the differences between Python and JavaScript with examples.'
]

for model_name, model_path in model_paths.items():
    if model_name not in servers:
        print(f'\n?? Skipping {model_name} - server not running')
        continue
    
    print(f'\n?? Running benchmarks for: {model_name}')
    
    client = LlamaCppClient(base_url='http://127.0.0.1:8090')
    bench_times = []
    bench_speeds = []
    
    for i, p in enumerate(prompts, 1):
        t0 = time.time()
        r = client.chat.create(
            messages=[{'role': 'user', 'content': p}],
            max_tokens=80,
            temperature=0.7,
        )
        dt = time.time() - t0
        tk = r.usage.completion_tokens if r.usage else 0
        sp = tk/dt if dt>0 else 0
        bench_times.append(dt)
        bench_speeds.append(sp)
        print(f'   {i}. Prompt: {p[:40]}... | {dt:.2f}s | {sp:.1f} tok/s')
    
    avg_time = sum(bench_times) / len(bench_times)
    avg_speed = sum(bench_speeds) / len(bench_speeds)
    
    benchmark_results[model_name] = {
        'avg_time': avg_time,
        'avg_speed': avg_speed,
        'bench_times': bench_times,
        'bench_speeds': bench_speeds
    }
    
    print(f'   Average: {avg_time:.2f}s | {avg_speed:.1f} tok/s')

?? STEP 9: PERFORMANCE BENCHMARKS

?? Running benchmarks for: Llama-3.2-3B-Instruct
   1. Prompt: Hello... | 0.22s | 37.0 tok/s
   2. Prompt: What is the capital of France?... | 0.12s | 67.0 tok/s
   3. Prompt: Write a paragraph about machine learning... | 1.09s | 73.7 tok/s
   4. Prompt: Explain the differences between Python a... | 1.10s | 72.6 tok/s
   Average: 0.63s | 62.5 tok/s

?? Running benchmarks for: Mistral-7B-Instruct
   1. Prompt: Hello... | 0.13s | 60.7 tok/s
   2. Prompt: What is the capital of France?... | 0.12s | 68.2 tok/s
   3. Prompt: Write a paragraph about machine learning... | 1.09s | 73.4 tok/s
   4. Prompt: Explain the differences between Python a... | 1.09s | 73.2 tok/s
   Average: 0.61s | 68.9 tok/s

?? Running benchmarks for: Phi-3-mini-4k
   1. Prompt: Hello... | 0.13s | 59.6 tok/s
   2. Prompt: What is the capital of France?... | 0.12s | 67.8 tok/s
   3. Prompt: Write a paragraph about machine learning... | 1.09s | 73.3 tok/s
   4. Prompt: Explain the diff

In [15]:
# Step 10: Streaming Test with Enhanced Debugging
print('='*70)
print('?? STEP 10: STREAMING TEST WITH DEBUGGING')
print('='*70)

for model_name, model_path in model_paths.items():
    if model_name not in servers:
        print(f'\n?? Skipping {model_name} - server not running')
        continue
    
    print(f'\n?? Testing streaming for: {model_name}')
    
    client = LlamaCppClient(base_url='http://127.0.0.1:8090')
    stream_prompt = 'What are the benefits of streaming?'
    
    try:
        print(f'Prompt: "{stream_prompt}"')
        print('Streaming response with debugging:')
        
        stream_resp = client.chat.create(
            messages=[{'role': 'user', 'content': stream_prompt}],
            max_tokens=80,
            temperature=0.7,
            stream=True
        )
        
        out = ''
        chunk_count = 0
        first_chunk_printed = False
        
        for chunk in stream_resp:
            chunk_count += 1
            
            # Print first chunk structure for debugging
            if not first_chunk_printed:
                print(f"\n  First chunk structure:")
                print(f"  Type: {type(chunk)}")
                if isinstance(chunk, dict):
                    print(f"  Keys: {list(chunk.keys())}")
                    # Print first level values
                    for key, value in chunk.items():
                        if key != 'choices':  # Don't print full choices if it's large
                            print(f"    {key}: {type(value)} - {str(value)[:100]}")
                        else:
                            print(f"    {key}: {type(value)} - length {len(value) if isinstance(value, list) else 'N/A'}")
                            if isinstance(value, list) and value:
                                print(f"      First choice type: {type(value[0])}")
                                if isinstance(value[0], dict):
                                    print(f"      First choice keys: {list(value[0].keys())}")
                first_chunk_printed = True
            
            # Extract content based on actual structure
            if isinstance(chunk, dict):
                # Check various possible structures
                content = None
                
                # Structure 1: OpenAI compatible
                if 'choices' in chunk and isinstance(chunk['choices'], list) and chunk['choices']:
                    choice = chunk['choices'][0]
                    if isinstance(choice, dict):
                        if 'delta' in choice and isinstance(choice['delta'], dict) and 'content' in choice['delta']:
                            content = choice['delta']['content']
                        elif 'message' in choice and isinstance(choice['message'], dict) and 'content' in choice['message']:
                            content = choice['message']['content']
                        elif 'text' in choice:
                            content = choice['text']
                
                # Structure 2: Direct content
                elif 'content' in chunk:
                    content = chunk['content']
                
                # Structure 3: Text field
                elif 'text' in chunk:
                    content = chunk['text']
                
                if content and content.strip():
                    out += content
                    print(content, end='', flush=True)
        
        print()  # New line after streaming
        
        if out:
            print(f'\n?? Streaming SUCCESS! Total chunks: {chunk_count}')
            print(f'   Response length: {len(out)} characters')
            print(f'   Full response: {out}')
        else:
            print(f'\n?? No content extracted from {chunk_count} chunks')
            
    except Exception as e:
        print(f'?? Streaming error for {model_name}: {e}')
        import traceback
        traceback.print_exc()

?? STEP 10: STREAMING TEST WITH DEBUGGING

?? Testing streaming for: Llama-3.2-3B-Instruct
Prompt: "What are the benefits of streaming?"
Streaming response with debugging:

  First chunk structure:
  Type: <class 'dict'>
  Keys: ['choices', 'created', 'id', 'model', 'system_fingerprint', 'object']
    choices: <class 'list'> - length 1
      First choice type: <class 'dict'>
      First choice keys: ['finish_reason', 'index', 'delta']
    created: <class 'int'> - 1770279282
    id: <class 'str'> - chatcmpl-OJNE9WPxmdFpQsDS6XjHbI02Tio3aoPF
    model: <class 'str'> - Llama-3.2-3B-Instruct-Q4_K_M.gguf
    system_fingerprint: <class 'str'> - b1-9f682fb
    object: <class 'str'> - chat.completion.chunk
Streaming has numerous benefits, including:

1. **Convenience**: Streaming allows users to access content from anywhere, at any time, as long as they have an internet connection.
2. **Cost-effective**: Streaming services often offer free or low-cost options for new users, and many subscriptio

In [16]:
# Step 11: Stop servers for each model
print('='*70)
print('?? STEP 11: STOP SERVERS')
print('='*70)

for model_name, server in servers.items():
    print(f'\n?? Stopping server for: {model_name}')
    try:
        server.stop_server()
        print(f'? Server stopped successfully')
        time.sleep(2)  # Give time for cleanup
    except Exception as e:
        print(f'? Error stopping server: {e}')

?? STEP 11: STOP SERVERS

?? Stopping server for: Llama-3.2-3B-Instruct
? Server stopped successfully

?? Stopping server for: Mistral-7B-Instruct
? Server stopped successfully

?? Stopping server for: Phi-3-mini-4k
? Server stopped successfully
